In [ ]:
import numpy as np
import pandas as pd
import gc
final =  pd.read_parquet('/kaggle/input/hw1parquets/hw13new.parquet')


# gonna cut unimportant features -> 
high_float_cols = [
        'AskExecRate500ms', 'BidExecRate500ms', 'timeBidExecute500ms',
       'AskExecRate1500ms', 'BidExecRate1500ms', 'timeBidExecute1500ms',
       'AskExecRate2500ms', 'BidExecRate2500ms', 'timeBidExecute2500ms',
       'AskExecRate5000ms', 'BidExecRate5000ms', 'timeBidExecute5000ms',]

float_cols = ['100msTradeImb_x', '250msTradeImb_x', '500msTradeImb_x',
       '1000msTradeImb_x', '2000msTradeImb_x', '100msTradeImb_y',
       '250msTradeImb_y', '500msTradeImb_y', '1000msTradeImb_y',
       '2000msTradeImb_y', 'PastRet100ms_x', 'PastRet250ms_x',
       'PastRet500ms_x', 'PastRet1000ms_x', 'PastRet2000ms_x',
       'PastRet250ms_y', 'PastRet500ms_y', 'PastRet1000ms_y',
       'PastRet2000ms_y', 'Div5S', 'Div9S', 'Div19S', 'Div38S', 'Div75S',
       'Div150S', 'Div300S', 'Div600S',  'spread',
       'RelVol6H', 'RelVol3H', 'RelVol1H', 'RelVol15m', 'RelVol5m', 'RelVol1m',
       'NewAskDiff', 'NewBidDiff', 'buy_amount', 'sell_amount', 'total_amount',
       
       '250msStd', '500msStd', '1000msStd', '2500msStd']

int_cols = ['time_since_price_change', 'hour', 'part_of_day']

final[float_cols] = final[float_cols].astype('float16')

final.drop(columns=['250msStd', 'RelVol1H', 'RelVol15m', 'spread', 'NewBidDiff', 'NewAskDiff', 'total_amount'], inplace=True)



trades = pd.read_parquet('/kaggle/input/hw1parquets/Hw2featuresNew.parquet')
trades = trades[['remote_ts2', 'ADL', 'ADX', 'CMO','MOM', 'ROC', 'StochRSI', 'RealizedVol', 'RealizedKernel']]
trades[['ADL', 'ADX', 'CMO','MOM', 'ROC', 'StochRSI', 'RealizedVol', 'RealizedKernel']] = trades[['ADL', 'ADX', 'CMO','MOM', 'ROC', 'StochRSI', 'RealizedVol', 'RealizedKernel']].astype('float16')

gc.collect()
# print(trades.head(10000).tail(10))

tc = trades.columns[2:]
trades[tc] = trades[tc].astype('float16')
trades.replace([np.inf, -np.inf], 0, inplace=True)
gc.collect()
trades.head(10000).tail()


trades.dropna(inplace=True)

final = pd.concatenate([final, trades[-len(final):]], axis=1)


final_columns = final.columns[1:]
y = final['target']
final.drop(columns=['target'], inplace=True)
# final=final.to_numpy()
gc.collect()

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler(copy=False)

X_train, X_test, y_train, y_test = train_test_split(final, y, train_size=0.45, test_size=0.35, random_state=42, shuffle=False)
l = len(final)
del final
del y

X_train[np.isinf(X_train)] = np.iinfo(np.int16).max
X_test[np.isinf(X_test)] = np.iinfo(np.int16).max
gc.collect()
X_train = scaler.fit_transform(X_train)
X_test  = scaler.transform(X_test)
# X_val = scaler.transform(X_val)


In [2]:
from catboost import CatBoostClassifier, Pool
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint as sp_randint
from scipy.stats import uniform as sp_uniform
import numpy as np

train_pool = Pool(X_train, y_train)
test_pool = Pool(X_test, y_test)


# Define the hyperparameter grid
param_dist = {
#     'learning_rate': sp_uniform(loc=0.01, scale=0.29),
    'depth': sp_randint(6, 17),
    'l2_leaf_reg': sp_uniform(loc=1e-6, scale=10),
#     'bagging_temperature': sp_uniform(loc=0, scale=10),
    'border_count': sp_randint(32, 256),
    'iterations': sp_randint(100,110),
    'boosting_type': ['Plain'],
}

# Create a CatBoost classifier
model = CatBoostClassifier(
    loss_function='MultiClass',
    eval_metric='MultiClass', 
    random_seed=42,
    colsample_bylevel=0.098,
    subsample=0.95,
#     l2_leaf_reg=9,
#     min_data_in_leaf=20000,
    bootstrap_type="Bernoulli", 
#     max_bin=187,
    random_strength=1,
)

randomized_search_result = model.randomized_search(param_dist, X_train, y_train, cv=2,
                  n_iter=10)

print(randomized_search_result['params'])
print('')
print(randomized_search_result['cv_results'])

# Evaluate the best model on the test set
# best_clf = random_search.best_estimator_
y_train_best = model.predict(train_pool)
y_test_best = model.predict(test_pool)



from sklearn.metrics import confusion_matrix

def metric(y_true, y_pred):
    conf_matrix = confusion_matrix(y_true, y_pred)
    weight_matrix = np.array([
        [1.9,     0,    -2],
        [-0.3,    0,    -0.3],
        [-2,      0,     1.9]
    ])
    
    hit_matrix = conf_matrix * weight_matrix
    hit_matrix_sum = np.sum(hit_matrix)
    action_count = np.sum(conf_matrix[:, 0]) + np.sum(conf_matrix[:, 2])
    if action_count == 0:
        return 0
    else:
        return hit_matrix_sum / np.sqrt(action_count)

train_score = metric(y_train, y_train_best)
test_score = metric(y_test, y_test_best)
print("Train / Test set score: ", train_score, test_score)


0:	learn: 1.0804291	test: 1.0804660	best: 1.0804660 (0)	total: 2.68s	remaining: 4m 49s
1:	learn: 1.0635666	test: 1.0636306	best: 1.0636306 (1)	total: 5.37s	remaining: 4m 47s
2:	learn: 1.0477986	test: 1.0478976	best: 1.0478976 (2)	total: 8.04s	remaining: 4m 44s
3:	learn: 1.0329192	test: 1.0330452	best: 1.0330452 (3)	total: 10.6s	remaining: 4m 37s
4:	learn: 1.0185638	test: 1.0187262	best: 1.0187262 (4)	total: 13.1s	remaining: 4m 32s
5:	learn: 1.0070490	test: 1.0072278	best: 1.0072278 (5)	total: 13.9s	remaining: 3m 58s
6:	learn: 0.9950784	test: 0.9952767	best: 0.9952767 (6)	total: 16.4s	remaining: 3m 58s
7:	learn: 0.9837020	test: 0.9839293	best: 0.9839293 (7)	total: 18.7s	remaining: 3m 56s
8:	learn: 0.9724329	test: 0.9726902	best: 0.9726902 (8)	total: 21.5s	remaining: 3m 59s
9:	learn: 0.9633114	test: 0.9635818	best: 0.9635818 (9)	total: 22.4s	remaining: 3m 41s
10:	learn: 0.9535989	test: 0.9538910	best: 0.9538910 (10)	total: 25.1s	remaining: 3m 43s
11:	learn: 0.9441510	test: 0.9444746	best

In [5]:
print(randomized_search_result['params'])

{'border_count': 178, 'depth': 14.0, 'l2_leaf_reg': 5.816403904771138, 'iterations': 108.0, 'boosting_type': 'Plain'}


In [14]:
model.get_all_params()

{'nan_mode': 'Min',
 'eval_metric': 'MultiClass',
 'iterations': 108,
 'sampling_frequency': 'PerTree',
 'leaf_estimation_method': 'Newton',
 'grow_policy': 'SymmetricTree',
 'penalties_coefficient': 1,
 'boosting_type': 'Plain',
 'model_shrink_mode': 'Constant',
 'feature_border_type': 'GreedyLogSum',
 'bayesian_matrix_reg': 0.10000000149011612,
 'eval_fraction': 0,
 'force_unit_auto_pair_weights': False,
 'l2_leaf_reg': 5.816403865814209,
 'random_strength': 1,
 'rsm': 0.09799999743700027,
 'boost_from_average': False,
 'model_size_reg': 0.5,
 'pool_metainfo_options': {'tags': {}},
 'subsample': 0.949999988079071,
 'use_best_model': False,
 'class_names': [-1, 0, 1],
 'random_seed': 42,
 'depth': 14,
 'posterior_sampling': False,
 'border_count': 178,
 'classes_count': 0,
 'auto_class_weights': 'None',
 'sparse_features_conflict_fraction': 0,
 'leaf_estimation_backtracking': 'AnyImprovement',
 'best_model_min_trees': 1,
 'model_shrink_rate': 0,
 'min_data_in_leaf': 1,
 'loss_function

In [ ]:
from sklearn.metrics import confusion_matrix

def metric(y_true, y_pred):
    conf_matrix = confusion_matrix(y_true, y_pred)
    weight_matrix = np.array([
        [1.9,     0,    -2],
        [-0.3,    0,    -0.3],
        [-2,      0,     1.9]
    ])
    
    hit_matrix = conf_matrix * weight_matrix
    hit_matrix_sum = np.sum(hit_matrix)
    action_count = np.sum(conf_matrix[:, 0]) + np.sum(conf_matrix[:, 2])
    if action_count == 0:
        return 0
    else:
        return hit_matrix_sum / np.sqrt(action_count)

In [16]:
y_train_best = model.predict(train_pool)
y_test_best = model.predict(test_pool)


train_score = metric(y_train, y_train_best)
test_score = metric(y_test, y_test_best)
print("Train / Test set score: ", train_score, test_score)


Train / Test set score:  1215.4479796287726 52.515681411795775


Либо это оттого что модель переобучилась тупо, либо от того что я срезал сильно наш трейн и тупо распределения оченль сильно разлчиаются. Попробую взять +- те параметры которые получились, но сделать шаг в стороно бОльшей регуляризации.

In [17]:
print(randomized_search_result['params'])

{'border_count': 178, 'depth': 14.0, 'l2_leaf_reg': 5.816403904771138, 'iterations': 108.0, 'boosting_type': 'Plain'}


In [ ]:
from catboost import CatBoostClassifier, Pool
import numpy as np

train_pool = Pool(X_train, y_train)
test_pool = Pool(X_test, y_test)

model_final = CatBoostClassifier(
    loss_function='MultiClass',
    eval_metric='MultiClass', 
    random_seed=42,
    colsample_bylevel=0.055,
    subsample=0.7,
    depth=7,
    l2_leaf_reg=10,
    min_data_in_leaf=20000,
    bootstrap_type="Bernoulli", 
    boosting_type = 'Plain',
    iterations=100,
    random_strength=1,
    verbose=10
)

model_final.fit(train_pool)

y_train_pred = model_final.predict(train_pool)
y_test_pred = model_final.predict(test_pool)


train_score = metric(y_train, y_train_pred)
test_score = metric(y_test, y_test_pred)
print("Train / Test set score: ", train_score, test_score)

In [ ]:
from catboost import CatBoostClassifier, Pool
import numpy as np

train_pool = Pool(X_train, y_train)
test_pool = Pool(X_test, y_test)

model_final = CatBoostClassifier(
    loss_function='MultiClass',
    eval_metric='MultiClass', 
    random_seed=42,
    colsample_bylevel=0.05,
    subsample=0.7,
    max_depth=7,
    l2_leaf_reg=10,
    min_data_in_leaf=20000,
    bootstrap_type="Bernoulli", 
    boosting_type = 'Plain',
    iterations=200,
    random_strength=1,
    verbose=10
)

model_final.fit(train_pool)

y_train_pred = model_final.predict(train_pool)
y_test_pred = model_final.predict(test_pool)


train_score = metric(y_train, y_train_pred)
test_score = metric(y_test, y_test_pred)
print("Train / Test set score: ", train_score, test_score)

In [ ]:
from catboost import CatBoostClassifier, Pool
import numpy as np

train_pool = Pool(X_train, y_train)
test_pool = Pool(X_test, y_test)

model_final = CatBoostClassifier(
    loss_function='MultiClass',
    eval_metric='MultiClass', 
    random_seed=42,
    colsample_bylevel=0.045,
    subsample=0.7,
    max_depth=7,
    l2_leaf_reg=10,
    min_data_in_leaf=20000,
    bootstrap_type="Bernoulli", 
    boosting_type = 'Plain',
    iterations=400,
    random_strength=1,
    verbose=10
)

model_final.fit(train_pool)

y_train_pred = model_final.predict(train_pool)
y_test_pred = model_final.predict(test_pool)


train_score = metric(y_train, y_train_pred)
test_score = metric(y_test, y_test_pred)
print("Train / Test set score: ", train_score, test_score)

### Бейзлайн так сказать

In [4]:
model_final = CatBoostClassifier(
    loss_function='MultiClass',
    eval_metric='MultiClass', 
    random_seed=42,
    bootstrap_type="Bernoulli", 
    boosting_type = 'Plain',
    iterations=50,
    verbose=10
)

model_final.fit(train_pool)

y_train_pred = model_final.predict(train_pool)
y_test_pred = model_final.predict(test_pool)


train_score = metric(y_train, y_train_pred)
test_score = metric(y_test, y_test_pred)
print("Train / Test set score: ", train_score, test_score)

Learning rate set to 0.5
0:	learn: 0.9798029	total: 12.2s	remaining: 9m 59s
10:	learn: 0.9101804	total: 1m 38s	remaining: 5m 49s
20:	learn: 0.9002238	total: 3m 2s	remaining: 4m 11s
30:	learn: 0.8916778	total: 4m 24s	remaining: 2m 41s
40:	learn: 0.8842038	total: 5m 52s	remaining: 1m 17s
49:	learn: 0.8781910	total: 7m 3s	remaining: 0us
Train / Test set score:  1034.3389266312652 224.63921142573574


# Postproccesing

In [7]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression(C=0.1, max_iter=25)
model.fit(X_train, y_train)

/opt/conda/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression(C=0.1, max_iter=25)

In [9]:
print("Метрика до постпроцессинга")
y_pred = model.predict(X_test)
print(metric(y_test, y_pred))

Метрика до постпроцессинга
170.0185955095964


In [8]:
import gc
y_train_probs = model.predict_proba(X_train)
y_test_probs = model.predict_proba(X_test)

# TODO NEXT
tsNeg = np.arange(0.32, 0.46, 0.025)
tsPos = np.arange(0.32, 0.46, 0.025)
best_ts = []
best_t = 0
for tn in tsNeg:
    for tp in tsPos:
        gc.collect()
        y_pred_train = np.zeros(y_train_probs.shape[0])
        y_pred_test = np.zeros(y_test_probs.shape[0])
        y_pred_train[(y_train_probs[:, 2] >= tp)] = 1
        y_pred_train[(y_train_probs[:, 2] < tp) & (y_train_probs[:, 0] >= tn)] = -1
        y_pred_train[(y_train_probs[:, 0] < tn) & (y_train_probs[:, 2] < tp)] = 0
        metric_train = metric(y_train, y_pred_train)
            
        best_m = metric_train
        y_pred_test[(y_test_probs[:, 2] >= tp)] = 1
        y_pred_test[(y_test_probs[:, 2] < tp) & (y_test_probs[:, 0] >= tn)] = -1
        y_pred_test[(y_test_probs[:, 0] < tn) & (y_test_probs[:, 2] < tp)] = 0
        metric_test = metric(y_test, y_pred_test)
        diff = metric_train-metric_test
        print("ths: ", round(tn, 3), round(tp, 3), " Metric train and test and diff: ", round(metric_train, 2), round(metric_test, 2), round(diff, 2))
        if metric_train > best_t:
            best_t = metric_train
            best_p = (tn, tp)
            best_train = metric_train
            best_test = metric_test
            
print("______________________________")
print("Best ths:", best_p)
print("Best metrics: ", best_train, best_test)

ths:  0.32 0.32  Metric train and test and diff:  189.88 237.33 -47.44
ths:  0.32 0.345  Metric train and test and diff:  272.07 214.78 57.29
ths:  0.32 0.37  Metric train and test and diff:  309.36 169.8 139.56
ths:  0.32 0.395  Metric train and test and diff:  319.37 146.0 173.37
ths:  0.32 0.42  Metric train and test and diff:  306.46 127.08 179.39
ths:  0.32 0.445  Metric train and test and diff:  289.39 106.16 183.24
ths:  0.345 0.32  Metric train and test and diff:  196.54 250.08 -53.54
ths:  0.345 0.345  Metric train and test and diff:  293.13 244.57 48.56
ths:  0.345 0.37  Metric train and test and diff:  346.01 211.2 134.81
ths:  0.345 0.395  Metric train and test and diff:  368.14 195.05 173.1
ths:  0.345 0.42  Metric train and test and diff:  365.48 181.88 183.61
ths:  0.345 0.445  Metric train and test and diff:  352.84 163.22 189.62
ths:  0.37 0.32  Metric train and test and diff:  177.84 254.6 -76.75
ths:  0.37 0.345  Metric train and test and diff:  279.99 256.38 23.61
t

Я не понял почему у меня в конце принтится не самое лучшее по трейну значение порогов. Но в общем видно, что если посмотреть глазами, то самые хорошие значения трейна так чтобы тест не сильно проседал в районе 0.32-0.37 для обоих порогов.

**Второй раунд. Более точный.**

In [12]:
import gc
y_train_probs = model.predict_proba(X_train)
y_test_probs = model.predict_proba(X_test)

# TODO NEXT
tsNeg = np.arange(0.36, 0.38, 0.01)
tsPos = np.arange(0.40, 0.43, 0.01)
best_ts = []
best_t = 0
for tn in tsNeg:
    for tp in tsPos:
        gc.collect()
        y_pred_train = np.zeros(y_train_probs.shape[0])
        y_pred_test = np.zeros(y_test_probs.shape[0])
        y_pred_train[(y_train_probs[:, 2] >= tp)] = 1
        y_pred_train[(y_train_probs[:, 2] < tp) & (y_train_probs[:, 0] >= tn)] = -1
        y_pred_train[(y_train_probs[:, 0] < tn) & (y_train_probs[:, 2] < tp)] = 0
        metric_train = metric(y_train, y_pred_train)
            
        best_m = metric_train
        y_pred_test[(y_test_probs[:, 2] >= tp)] = 1
        y_pred_test[(y_test_probs[:, 2] < tp) & (y_test_probs[:, 0] >= tn)] = -1
        y_pred_test[(y_test_probs[:, 0] < tn) & (y_test_probs[:, 2] < tp)] = 0
        metric_test = metric(y_test, y_pred_test)
        diff = metric_train-metric_test
        print("ths: ", round(tn, 3), round(tp, 3), " Metric train and test and diff: ", round(metric_train, 2), round(metric_test, 2), round(diff, 2))
        if metric_train > best_t:
            best_t=metric_train
            best_p= (tn, tp)
            best_train = metric_train
            best_test = metric_test
            
print("______________________________")
print("Best ths:", best_p)
print("Best metrics: ", best_train, best_test)

ths:  0.36 0.4  Metric train and test and diff:  372.71 214.47 158.24
ths:  0.36 0.41  Metric train and test and diff:  373.17 209.15 164.02
ths:  0.36 0.42  Metric train and test and diff:  378.37 205.43 172.94
ths:  0.37 0.4  Metric train and test and diff:  368.06 223.68 144.38
ths:  0.37 0.41  Metric train and test and diff:  370.01 219.24 150.76
ths:  0.37 0.42  Metric train and test and diff:  376.41 215.66 160.74
ths:  0.38 0.4  Metric train and test and diff:  366.28 231.85 134.43
ths:  0.38 0.41  Metric train and test and diff:  369.5 227.77 141.72
ths:  0.38 0.42  Metric train and test and diff:  376.75 225.18 151.58
______________________________
Best ths: (0.36, 0.42000000000000004)
Best metrics:  378.3682221466522 205.42884216722362


Нашел баг с выводом. Ладно. Видим что лучше всего значения метрики при 0.34 и 0.37 тк так у нас меньше всего разрыв между трейном и тестом, а значит более надежные цифры. 

# Custom loss

In [ ]:
def custom_regression_loss(preds: Tensor, targets: Tensor) -> Tensor:
    """ penalize undershooting much more than overshooting """
    raw_diff = preds - targets
    is_undershoot = raw_diff < 0
    loss = torch.zeros_like(raw_diff)
    loss[is_undershoot] = torch.abs(raw_diff[is_undershoot] - 1) ** 3 - 1
    loss[~is_undershoot] = torch.abs(raw_diff[~is_undershoot])
    loss = loss.sum()
    return loss

In [8]:
import torch

def custom_loss(y_pred, y_target):
    epsilon=1e-5
    y_pred = np.clip(y_pred, epsilon, 1. - epsilon)
    
    weight_matrix = np.array([
        [1.9,     0,    -2],
        [-0.3,    0,    -0.3],
        [-2,      0,     1.9]
    ])
    
    I_neg = (y_target == -1)
    I_zero = (y_target == 0)
    I_pos = (y_target == 1)
    
    
    N = y_pred.shape[0]
    
    ce_1 = torch.nn.functional.cross_entropy(y_pred[I_neg], y_target[I_neg], weight_matrix[0])
    ce_2 = torch.nn.functional.cross_entropy(y_pred[I_neg], y_target[I_neg], weight_matrix[1])
    ce_3 = torch.nn.functional.cross_entropy(y_pred[I_neg], y_target[I_neg], weight_matrix[2])
    
    return ce_1 + ce_2 + ce_3

In [3]:
weight_matrix = np.array([
            [1.9, -0.3, -2],
            [-0.3, 0, -0.3],
            [-2, -0.3, 1.9]
])


class MultiClassObjective(object):
    def calc_ders_multi(self, approx, target, weight):
        approx = np.array(approx) - max(approx)
        exp_approx = np.exp(approx)
        exp_sum = exp_approx.sum()
        grad = []
        hess = []
        for j in range(len(approx)):
            der1 = -exp_approx[j] / exp_sum
            if j == target:
                der1 += 1
            hess_row = []
            for j2 in range(len(approx)):
                der2 = exp_approx[j] * exp_approx[j2] / (exp_sum**2)
                if j2 == j:
                    der2 -= exp_approx[j] / exp_sum
                hess_row.append(der2 * weight)

            grad.append(der1 * weight)
            hess.append(hess_row)

        return (grad, hess)



In [3]:
class MultiClassObjective(object):
    def calc_ders_multi(self, approx, target, weight_matrix):
        weight_matrix = np.array([
            [1.9, -0.3, -2],
            [-0.3, 0, -0.3],
            [-2, -0.3, 1.9]
        ])
        approx = np.array(approx) - max(approx)
        exp_approx = np.exp(approx)
        exp_sum = exp_approx.sum()
        grad = []
        hess = []
        for j in range(len(approx)):
            der1 = -exp_approx[j] / exp_sum
            if j == target:
                der1 += 1
            hess_row = []
            for j2 in range(len(approx)):
                der2 = exp_approx[j] * exp_approx[j2] / (exp_sum**2)
                if j2 == j:
                    der2 -= exp_approx[j] / exp_sum
                hess_row.append(der2 * weight_matrix[target][j2])

            grad.append(der1 * weight_matrix[target][j])
            hess.append(hess_row)

        return (np.array(grad), np.array(hess))


In [4]:
class MultiClassObjective(object):
    def calc_ders_range(self, approx, target, weight):
        approx = np.array(approx) - max(approx)
        exp_approx = np.exp(approx)
        exp_sum = exp_approx.sum()
        grad = []
        hess = []
        for j in range(len(approx)):
            der1 = -exp_approx[j] / exp_sum
            if j == target:
                der1 += 1
            hess_row = []
            for j2 in range(len(approx)):
                der2 = exp_approx[j] * exp_approx[j2] / (exp_sum**2)
                if j2 == j:
                    der2 -= exp_approx[j] / exp_sum
                hess_row.append(der2 * weight)

            grad.append(der1 * weight)
            hess.append(hess_row)

        return (grad, hess)

In [3]:
from catboost import CatBoostClassifier, Pool
import numpy as np

train_pool = Pool(X_train, y_train)
test_pool = Pool(X_test, y_test)

In [ ]:
model_custom = CatBoostClassifier(
    max_depth=7,
    loss_function=MultiClassObjective(),
    eval_metric=MultiClassObjective(),
    iterations=10,
#     subsample=0.8,
#     depth=7,
    l2_leaf_reg=10
)

model_custom.fit(train_pool, verbose=10)

y_train_custom = model_custom.predict(train_pool)
y_test_custom = model_custom.predict(test_pool)

train_score = metric(y_train, y_train_custom)
test_score = metric(y_test, y_test_custom)
print("Train / Test set score: ", train_score, test_score)